# [운동 동작 분류 AI - Private 5위](https://dacon.io/competitions/official/235689/codeshare/2387?page=2&dtype=recent&ptype=pub)


ML : LGBM + XGB + EXT ~ Voting Classifier  
DL : CBAM 

In [6]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


from itertools import combinations
import math
from sklearn.preprocessing import PowerTransformer, MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler,OneHotEncoder 
from tqdm import tqdm

import shap
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import cross_val_score
from scipy.ndimage import gaussian_filter1d

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.cluster import KMeans
from scipy.stats import skew, kurtosis


from sklearn.metrics import mean_squared_error, log_loss, make_scorer
from sklearn.model_selection import RandomizedSearchCV, train_test_split, StratifiedKFold, GridSearchCV

import time
from bayes_opt import BayesianOptimization
from vecstack import stacking 
from datetime import datetime 

from scipy.stats import gmean

In [12]:
train_x = pd.read_csv('../workout/train_features.csv')
test = pd.read_csv('../workout/test_features.csv')
y = pd.read_csv('../workout/train_labels.csv')

In [9]:
pd.set_option('display.max_columns', None)

In [14]:
train_x.head(2)

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216


## Feature Gerneration 

데이터에 방향성으로 속도 조합하기 

In [46]:
acc_col = train_x.columns[2:5]
gy_col = train_x.columns[5:]


for target_list in [acc_col, gy_col]:
    for a,b in combinations(target_list,2):
        columns_name = a+b[-1]
        
        train_x[columns_name] = (train_x[a]**2 + train_x[b]**2) ** (1/2)
    

In [47]:
train_x.head(3)

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_xy,acc_xz,acc_yz,gy_xy,gy_xz,gy_yz
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,1.219352,1.215188,0.232832,30.554737,31.681636,44.007023
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,1.302978,1.300557,0.269956,39.140277,24.929059,46.402968
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,1.319119,1.328987,0.319800,44.270593,25.279765,50.722604


In [48]:
train_x['acc_xyz'] = (train_x['acc_x']**2 + train_x['acc_y']**2 + train_x['acc_z']**2) ** (1/3)
train_x['gy_xyz'] = (train_x['gy_x']**2 + train_x['gy_y']**2 + train_x['gy_z']**2) ** (1/3)

In [49]:
for target_list in [acc_col, gy_col]:
    for a, b in combinations(target_list, 2):
        col_name = a + b[-1]
        
        test[col_name] = (test[a]**2 + test[b]**2 ) ** (1/2)
        
        
test['acc_xyz'] = (test['acc_x']**2 + test['acc_y']**2 + test['acc_z']**2) ** (1/3)
test['gy_xyz'] = (test['gy_x']**2 + test['gy_y']**2 + test['gy_z']**2 ) **(1/3)

### Pivot Table

In [51]:
agg_train_x = pd.pivot_table(train_x, values = train_x.columns[2:], index ='id', 
               aggfunc = ['sum', 'mean', 'mad',
                         'median','min','max',
                         'std','var',
                         'sem', 'skew','quantile']).reset_index() 

agg_train_x.columns  =  [i[0] +'_'+ i[1] for i in agg_train_x.columns]

In [52]:
agg_test_x = pd.pivot_table(test, values = test.columns[2:], index ='id', 
               aggfunc = ['sum', 'mean', 'mad',
                         'median','min','max',
                         'std','var',
                         'sem', 'skew','quantile']).reset_index() 

agg_test_x.columns  =  [i[0] +'_'+ i[1] for i in agg_test_x.columns]

In [54]:
print(agg_train_x.shape, agg_test_x.shape)

(3125, 155) (782, 155)


### Make Features with Logloss

In [59]:
train_x.shape

(1875000, 16)

In [60]:
grouped_train_x = train_x.iloc[:,2:].values.reshape(-1, 600, train_x.shape[1]-2)
grouped_test_x = test.iloc[:,2:].values.reshape(-1, 600, test.shape[1] -2 )

In [61]:
print(grouped_train_x.shape, grouped_test_x.shape)

(3125, 600, 14) (782, 600, 14)


In [64]:
times = []
for num in ['002','010','020','030','050','100','200','300','600' ]:
    
    target_time = float(num[0]+'.'+num[1:])
    # times.append(target_time)
    globals()['log_'+num+'_train_x'] =np.zeros((grouped_train_x.shape[0], 
                                               grouped_train_x.shape[1] - int(target_time//0.02),
                                              grouped_train_x.shape[2]))
    
# times